In [ ]:
# UDP Broadcast Client
# Run this in a separate Jupyter notebook (can run multiple instances)

import socket
import json
import datetime
import uuid
import time

# Create a unique ID for this client
client_id = str(uuid.uuid4())[:8]

# Create UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

# Enable broadcasting mode (optional for client but doesn't hurt)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

# Allow reuse of addresses (useful when running multiple clients on same machine)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Set a timeout so the socket will not block indefinitely when trying to receive data
client_socket.settimeout(10)

# Bind to the client port to receive broadcasts
client_port = 37020
client_socket.bind(('', client_port))  # Empty string means all interfaces

print(f"Client {client_id} started, listening on port {client_port}")
print("Press Ctrl+C to stop")

# Create a socket for sending responses back to the server
response_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
response_socket.settimeout(1)

# Store received messages for demonstration purposes
received_messages = []
last_processed_id = -1

try:
    while True:
        try:
            # Receive data from the server (blocking call with timeout)
            data, addr = client_socket.recvfrom(1024)
            server_ip = addr[0]
            server_port = addr[1]
            
            # Decode and parse the message
            message_str = data.decode('utf-8')
            message = json.loads(message_str)
            
            # Extract timestamp and state
            timestamp = message.get("timestamp", "unknown")
            state = message.get("state", "unknown")
            message_id = message.get("message_id", -1)
            
            # Only process new messages (in case of duplicate broadcasts)
            if message_id > last_processed_id:
                last_processed_id = message_id
                
                # Get current time to calculate latency
                receive_time = datetime.datetime.now().isoformat()
                
                # Print received message
                print(f"\nReceived broadcast from {server_ip}:{server_port}")
                print(f"Message ID: {message_id}")
                print(f"Timestamp: {timestamp}")
                print(f"State: {state}")
                
                # Append to message history
                received_messages.append({
                    "server_ip": server_ip,
                    "server_port": server_port,
                    "timestamp": timestamp,
                    "receive_time": receive_time,
                    "state": state,
                    "message_id": message_id
                })
                
                # Optionally send a response back to the server
                response = f"Client {client_id} received message {message_id}"
                response_socket.sendto(response.encode('utf-8'), (server_ip, server_port))
                
                # Print statistics
                print(f"Total messages received: {len(received_messages)}")
                
                # Display the last 5 states for demonstration
                if len(received_messages) >= 5:
                    recent_states = [msg["state"] for msg in received_messages[-5:]]
                    print(f"Last 5 states: {recent_states}")
        
        except socket.timeout:
            print(".", end="", flush=True)  # Print a dot to show we're still waiting
        
        except json.JSONDecodeError:
            print(f"Error: Received invalid JSON data: {message_str}")
        
        except Exception as e:
            print(f"Error: {e}")
            
except KeyboardInterrupt:
    print("\nClient shutting down...")

finally:
    # Clean up the sockets
    client_socket.close()
    response_socket.close()
    print("Client stopped")
    
    # Print summary
    print(f"\nSummary: Received {len(received_messages)} messages")